## Logon

In [6]:
import os 
import random
import pandas as pd

# read four dirs logon.csv
# weeks = range(1, 5)
weeks = range(1, 4)  # 1: change

week_dates = [5, 5, 5, 5, 5]
week_pointer = 0

# foundation_model = "gemini" # 0: change
foundation_model = "openai"

scenario = "Tech-Company" # 1: change
company_type = "tech" # 2: change
company_type_member = "tech_company" # 3: change

# scenario = "Finance-Company"
# company_type = "finance"
# company_type_member = "finance_corporation"

# scenario = "Medical-Institution"
# company_type_member = "medical_institution"
# company_type = "medical"

# Reading
log_root_dir = f'/data2/visitor/ASE25/Chimera-Dataset/'
root_dir = "/data2/visitor/ASE25/Chimera"

# concat all the logon.csv
logon_csvs = pd.DataFrame()

#### load all data ####
for week in weeks:
    weekly_dir = os.path.join(log_root_dir, scenario, f'week{week}-{foundation_model}-{company_type}')
    weekly_logon_log_path = os.path.join(weekly_dir, "execution_logs", 'logon.csv')

    df = pd.read_csv(weekly_logon_log_path, encoding='utf-8')
    print(f"Processing {weekly_logon_log_path} with {len(df)} rows")

    # Concatenate all DataFrames
    logon_csvs = pd.concat([logon_csvs, df], ignore_index=True)

print(f"Total rows in logon_csvs: {len(logon_csvs)}")

#### fix if instant logon and logout ####
for i in range(len(logon_csvs)):
    if i > 0:
        sim_time = logon_csvs.loc[i, 'sim_timestamp']
        sim_id = logon_csvs.loc[i, 'id']
        prev_id = logon_csvs.loc[i-1, 'id']
        if sim_id == prev_id:
            prev_sim_time = logon_csvs.loc[i-1, 'sim_timestamp']
            if sim_time == prev_sim_time:
                # random add near 5 minutes to sim_id
                new_time = pd.to_datetime(logon_csvs.loc[i, 'sim_timestamp'], format='%H:%M:%S') + pd.Timedelta(seconds=random.randint(30, 200))
                logon_csvs.loc[i, 'sim_timestamp'] = new_time.strftime('%H:%M:%S')

starting_date = "2025-05-01"

#### settle the sim timestamp with proper date ####
date_counter = 1
for i in range(len(logon_csvs)):
    sim_time = pd.to_datetime(logon_csvs.loc[i, 'sim_timestamp'], format='%H:%M:%S')
    
    if i == 0:
        sim_date = starting_date
    else:
        prev_time = pd.to_datetime(logon_csvs.loc[i-1, 'sim_timestamp'], format='%H:%M:%S')
        if (sim_time < prev_time) and ((prev_time - sim_time) > pd.Timedelta(hours=6)):
            if date_counter == week_dates[week_pointer]:
                # print("next week")
                add_date = 8 - week_dates[week_pointer]
                week_pointer += 1
                date_counter = 1
            else:
                date_counter += 1
                add_date = 1
            sim_date = (pd.to_datetime(sim_date) + pd.Timedelta(days=add_date)).strftime('%Y-%m-%d')
            # print(f"Date changed to {sim_date} at index {i}")
    logon_csvs.loc[i, 'full_sim_timestamp'] = f"{sim_date} {logon_csvs.loc[i, 'sim_timestamp']}"

Processing /data2/visitor/ASE25/Chimera-Dataset/Tech-Company/week1-openai-tech/execution_logs/logon.csv with 4074 rows
Processing /data2/visitor/ASE25/Chimera-Dataset/Tech-Company/week2-openai-tech/execution_logs/logon.csv with 7684 rows
Processing /data2/visitor/ASE25/Chimera-Dataset/Tech-Company/week3-openai-tech/execution_logs/logon.csv with 4183 rows
Total rows in logon_csvs: 15941


In [7]:
logon_csvs.head(5)

,id,real_timestamp,sim_timestamp,name,container_id,status,full_sim_timestamp
0,qtest-1,2025-05-16 15:11:09,07:49:44,Ethan Carter,7b8a9c0d1e2f,login,2025-05-01 07:49:44
1,tart-1,2025-05-16 15:11:11,07:50:15,Ayana Nakamura,5e7f8a9b0c1d,login,2025-05-01 07:50:15
2,3dmod-2,2025-05-16 15:11:14,07:50:59,Ricardo Oliveira,4567b8c9d0e1,login,2025-05-01 07:50:59
3,3dmod-1,2025-05-16 15:11:19,07:52:16,Fatima Al-Farsi,1234a5b6c7d8,login,2025-05-01 07:52:16
4,tart-1,2025-05-16 15:11:20,07:52:23,Ayana Nakamura,5e7f8a9b0c1d,logout,2025-05-01 07:52:23


In [8]:
output_dir = f"{root_dir}/Final-Output-OpenAI/{company_type_member}/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

sim_idx = logon_csvs.columns.get_loc("sim_timestamp")
full_sim = logon_csvs.pop("full_sim_timestamp")
logon_csvs.insert(sim_idx + 1, "full_sim_timestamp", full_sim)

# rename the column "name" to "user"
logon_csvs.rename(columns={'name': 'user'}, inplace=True)
# rename the column "full_sim_timestamp" to "time"
logon_csvs.rename(columns={'full_sim_timestamp': 'date'}, inplace=True)
# rename the column "container_id" to "pc"
logon_csvs.rename(columns={'container_id': 'pc'}, inplace=True)
# rename the column "status" to "activity"
logon_csvs.rename(columns={'status': 'activity'}, inplace=True)

# sorting by time
logon_csvs = logon_csvs.sort_values(by='date', ascending=True).reset_index(drop=True)

# save full data
logon_csvs.to_csv(f'{output_dir}/{company_type_member}_logon_full.csv', index=False, encoding='utf-8')

# remove the column "real_timestamp" and "sim_timestamp"
logon_csvs.drop(columns=['real_timestamp', 'sim_timestamp'], inplace=True) 

# save the email_csvs to csv
logon_csvs.to_csv(f'{output_dir}/{company_type_member}_logon.csv', index=False, encoding='utf-8')
# logon_csvs.to_csv(f'{output_dir}/{company_type}-logon-full.csv', index=False, encoding='utf-8')